In [1]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance
import os
from binance.spot import Spot

import pandas as pd
from dotenv import load_dotenv


In [2]:
# read from local .env file
load_dotenv() 

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']

spotClient = Spot()
# myClient = Client(api_key, api_secret, testnet=True)

In [3]:
# INPUT
## SET this to get d2, d3, d4
day_aggregation = 3

## ema (20 or 25)
ema_size = 25
symbol = "BTCUSDT"



In [4]:
# TIMEFRAME
## set timeframe to query binance, based on day_aggregation
do_manual_aggregation = True
timeframe = '1d'
if day_aggregation == 3:
    timeframe = '3d'
    do_manual_aggregation = False
elif day_aggregation == 7:
    timeframe = '1w'
    do_manual_aggregation = False
elif day_aggregation == 30:
    timeframe = '1m'
    do_manual_aggregation = False
        

In [5]:
# FUNCTIONS
## Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = spotClient.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

## Define function to calculate an arbitrary EMA      
def calc_generic_ema(dataframe, symbol, timeframe, ema_size):
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [6]:
# MAIN
raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=1000)
## Convert into Dataframe
df = pd.DataFrame(raw_data)

if do_manual_aggregation:
    df['d_index'] = df.index//day_aggregation
    # get high / low
    df['d_high'] = df.groupby('d_index')['high'].transform(max)
    df['d_low'] = df.groupby(['d_index'])['low'].transform(min)
    
    # get open / close
    
    df['d_open'] = df.groupby(['d_index'])['open'].transform('first')
    df['d_close'] = df.groupby(['d_index'])['close'].transform('last')

    df = df.groupby('d_index').first()
    
    # clean up d1 column
    del df['open']
    del df['high']
    del df['low']
    del df['close']
    
    # rename d_column
    df = df.rename(columns={"d_high": "high", "d_low": "low", "d_close": "close", "d_open": "open" })
    
df['time'] = pd.to_datetime(df['time'], unit='ms')
# ema
df = calc_generic_ema(df, symbol, timeframe, ema_size)

# clean up

del df['volume']
del df['close_time']
del df['quote_asset_volume']
del df['number_of_trades']
del df['taker_buy_base_asset_volume']
del df['taker_buy_quote_asset_volume']

# Calculate oversell
df['oversell_day'] = 0
df.loc[df['high'] > df['ema_25'], 'oversell_day'] = 1
df.loc[df['ema_25'] == 0, 'oversell_day'] = 0


df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
df.loc[df['oversell_day'] == 0, 'streak'] = 0


# display

print('Result')
df.tail(30)

Result


,time,open,high,low,close,ema_25,oversell_day,streak
651,2022-12-22,16824.68,16955.14,16559.85,16836.12,17850.414259,0,0
652,2022-12-25,16835.73,16972.83,16592.37,16706.36,17762.410085,0,0
653,2022-12-28,16706.06,16785.19,16333.00,16607.48,17673.569309,0,0
654,2022-12-31,16607.48,16799.23,16470.00,16672.87,17596.592439,0,0
655,2023-01-03,16672.78,16991.87,16605.28,16831.85,17537.766098,0,0
656,2023-01-06,16831.85,17176.99,16679.00,17127.83,17506.232552,0,0
657,2023-01-09,17127.83,18000.00,17104.66,17943.26,17539.850048,1,1
658,2023-01-12,17943.26,21258.00,17892.05,20954.92,17802.547737,1,2
659,2023-01-15,20952.76,21647.45,20551.01,21134.81,18058.875603,1,3
660,2023-01-18,21132.29,22755.93,20407.15,22667.21,18413.362864,1,4
